# State level Assembly Constituency Mapper

This notebook maps each state's master list details to the attribute table of asn AC shapefile, which will then enable the user to create state level AC identifier maps. These maps, would then inherit the features present in the masterlist and will provide a concise undertsanding of the current stage of data availability, presence of missing constitutencies, presenc eof image file for certain constituencies etc. This exercise is an intermediate priocess to assess the data generation process and its progress. This notebook doesn't serve to provide any data wrangling abilities or wrnagle any existing raw or interim data files.

In [1]:

#importing necessary libraries
import pandas as pd
from pathlib import Path
from fuzzywuzzy import process

In [7]:
#defining directories
dir_path=Path.cwd().parents[1]
masterlist_path=dir_path.joinpath('data_docs/golden_states_masterlist')
appended_masterlist=dir_path.joinpath('data_docs/appended_masterlist.xlsx')
attr_table_path=dir_path.joinpath('data/external/India_AC_shapefile.csv')


In [ ]:
def column_name_unifier(data_lists: list):
    """A function to capture column names of the masterlists and then unify them for a succesful data concatenation"""
    col_names = [x.columns for x in data_lists]

    col_names = pd.Series([y for x in col_names for y in x])
    col_names = col_names.str.lower().str.strip().str.replace(" ", "_")
    col_names = pd.Series(col_names.unique())

    updated_cols_dict = {
        "assembly_constituency": "ac_name",
        "sate": "state",
        "ac_no.": "ac_no",
        "error": "error_comment",
        "constituency": "ac_name",
        "ac": "ac_name",
        "ac.no": "ac_no",
        "comment": "note",
        "unnamed:_10":"note",
        "unnamed:_11":"note"
    } #unamed cols are present only in gujarat and jharkhand. The both are notes columns

    col_names = (
        col_names.replace(updated_cols_dict)
        .unique()
    )

    # attributing columsn to frame based ont he column size
    col_names = col_names[0:12]

    print(col_names)

    for df in data_lists:
        # try:
        df_cols = df.columns

        str_cleaner = lambda x: str(x).lower().strip().replace(" ", "_")

        df_cols = [
            updated_cols_dict[str_cleaner(x)]
            if str_cleaner(x) not in col_names
            else str_cleaner(x)
            for x in df_cols
        ]

        # except KeyError:
        #     print(df)
        #     for x in df_cols:
        #         if str_cleaner(x) in ["unnamed:_11, unnamed:_10"]:
        #             print(df)
        #             df.drop(str_cleaner(x), inplace=True)
        #             print(df)
        #             df_cols.remove(str_cleaner(x))

        df.columns=df_cols
        print(df_cols, df['state'].unique()[0])

    return data_lists

In [8]:
def master_reader():

    files=list(masterlist_path.glob("*"))[1:]
    print(files)
    dfs=[pd.read_excel(file, sheet_name=x, engine='openpyxl') for file in files for x in [0,1,2]]
    
    df=pd.concat(dfs, axis=0)

    df.to_excel(appended_masterlist, index=False)

    return df



In [9]:
# master id column creator
master=master_reader()
# master['state']=master['state'].str.lower().str.strip().str.replace(" ", "_")
# master['ac_name']=master['ac_name'].str.lower().str.strip().str.replace(" ", "_")
master

[WindowsPath('d:/work_isb/projects/Voting/data_docs/golden_states_masterlist/andhra_pradesh_masterlist.xlsx'), WindowsPath('d:/work_isb/projects/Voting/data_docs/golden_states_masterlist/assam_masterlist.xlsx'), WindowsPath('d:/work_isb/projects/Voting/data_docs/golden_states_masterlist/chhattisgarh_masterlist.xlsx'), WindowsPath('d:/work_isb/projects/Voting/data_docs/golden_states_masterlist/goa_masterlist.xlsx'), WindowsPath('d:/work_isb/projects/Voting/data_docs/golden_states_masterlist/gujarat_masterlist.xlsx'), WindowsPath('d:/work_isb/projects/Voting/data_docs/golden_states_masterlist/himachal_pradesh_masterlist.xlsx'), WindowsPath('d:/work_isb/projects/Voting/data_docs/golden_states_masterlist/jharkhand_masterlist.xlsx'), WindowsPath('d:/work_isb/projects/Voting/data_docs/golden_states_masterlist/karnataka_masterlist.xlsx'), WindowsPath('d:/work_isb/projects/Voting/data_docs/golden_states_masterlist/kerala_masterlist.xlsx'), WindowsPath('d:/work_isb/projects/Voting/data_docs/gol

d:\work_isb\projects\Voting\.venv\lib\site-packages\openpyxl\worksheet\_reader.py:335: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
d:\work_isb\projects\Voting\.venv\lib\site-packages\openpyxl\worksheet\_reader.py:335: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
d:\work_isb\projects\Voting\.venv\lib\site-packages\openpyxl\worksheet\_reader.py:335: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
d:\work_isb\projects\Voting\.venv\lib\site-packages\openpyxl\worksheet\_reader.py:335: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
d:\work_isb\projects\Voting\.venv\lib\site-packages\openpyxl\worksheet\_reader.py:335: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
d:\work_isb\projects\Voting\.venv\lib\site-packages\openpyxl\worksheet\_reader.py:335: UserWarning: Data Validation extension

,year,election,ac_no,state,ac_name,downloaded,converted,cleaned,image,indic,file_status,error_comment,bye_election,polling_station_name,note
0,2014,Vidhan Sabha,56,Andhra Pradesh,Achanta,1,0.0,0,1,0,Downloaded not converted,NaN,0,0,NaN
1,2014,Vidhan Sabha,105,Andhra Pradesh,Addanki,1,0.0,0,1,0,Downloaded not converted,NaN,0,0,NaN
2,2014,Vidhan Sabha,146,Andhra Pradesh,Adoni,1,1.0,1,1,0,file cleaned,NaN,0,0,NaN
3,2014,Vidhan Sabha,134,Andhra Pradesh,Allagadda,1,0.0,0,1,0,Downloaded not converted,NaN,0,0,NaN
4,2014,Vidhan Sabha,147,Andhra Pradesh,Alur,1,0.0,0,1,0,Downloaded not converted,NaN,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,2021,Vidhan Sabha,109,West Bengal,Khardaha,0,0.0,0,0,0,not downloaded,NaN,1,0,NaN
18,2021,Vidhan Sabha,127,West Bengal,Gosaba (SC),0,0.0,0,0,0,not downloaded,NaN,1,0,NaN
19,2021,Vidhan Sabha,7,West Bengal,Dinhata,0,0.0,0,0,0,not downloaded,NaN,1,0,NaN
20,2021,Vidhan Sabha,159,West Bengal,Bhabanipur,0,0.0,0,0,0,not downloaded,NaN,1,0,NaN


In [ ]:
shp_file=pd.read_csv(attr_table_path)
shp_cols=shp_file.columns
shp_cols=[x.lower() for x in shp_cols]
shp_file.columns=shp_cols

shp_file['st_name']=shp_file['st_name'].str.lower().str.strip().str.replace(" ", "_")
shp_file['ac_name']=shp_file['ac_name'].str.lower().str.strip().str.replace(" ", "_")
# print(shp_file['ac_name'].str.isalnum().value_counts())
shp_file[(shp_file['ac_name']=='')]

## State Mapper

In [ ]:


master['state']=master['state'].str.replace('telangana', 'andhra_pradesh')
master['state']=master['state'].str.replace('odisha', 'orissa')

shp_file['id']=shp_file['st_name'] + shp_file['ac_name']
master['id']=master['state']+master['ac_name']
len(shp_file['id'].unique())

# state_master=master['state'].unique()
# state_shp=shp_file['st_name'].unique()
# result = [
#     process.extractOne(i, state_shp)
#     for i in state_master
# ]

# results=[{match[0]:original} for match, original in zip(result,state_master)]


# print(results)




In [ ]:
shp_file[shp_file.duplicated(['id'], keep=False)].to_csv("dup_shapes.csv", index=False)

In [ ]:
shp_file[shp_file['id']=='sikkimkabi_lungchuk(bl)']

## AC Mapping

In [ ]:
# creating a list of matches for unmerged state_dist_block names
for state in shp_file['st_name'].unique():
    state_master=master[master['state'].isin(['odisha'])]
    state_shp=shp_file[shp_file['st_name'].isin(['orissa'])]
    result = [
        process.extractOne(i, state_shp['id'])
        for i in state_master['id']
    ]

    ac_dict=dict(zip(result,state_master['id']))


    print(ac_dict)

# # converting and editing the fuzzy matches to a dataframe
# result = pd.DataFrame(result, columns=["match", "score", "id"])
# result.drop("id", axis=1, inplace=True)

# result

# # creating a proxy dataframe with names of unmerged original name sof state_dist_block
# not_lgd_proxy_df = (
#     pd.DataFrame(not_lgd_mapped["state_dist_block"], index=None)
#     .reset_index()
#     .drop("index", axis=1)
# )

# # creating a dataframe for fuzzy mapper
# mapper_df = pd.concat(
#     [not_lgd_proxy_df, result],
#     axis=1,
#     ignore_index=True,
#     names=["original", "match", "score"],
# )
# mapper_df = mapper_df[mapper_df[2] >= 90]

# # creating a dictionary for fuzzy mapper
# mapper_dict = dict(zip(mapper_df[0], mapper_df[1]))

# # applying the mapper dictionary on the original processed state file to correct for fuzzy matched names
# df["state_dist_block"] = df["state_dist_block"].replace(mapper_dict)


In [ ]:
# master=master.rename(columns={
#     'ac_name':'ac_name_master'
# })
merged=pd.merge(shp_file, master, on='id', how='outer', indicator=True, validate='1:m')